<a href="https://colab.research.google.com/github/huynh0710hung/PEFT-LoRA-Model-Tunning/blob/main/Self_sup_Tunning_with_LoRa_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install neccessary libs
!pip install -q accelerate peft bitsandbytes transformers trl

In [2]:
import os
import re
import torch
import bitsandbytes
import accelerate
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    HfArgumentParser, TrainingArguments, pipeline, logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
# Checking and formating data set
# Org: https://huggingface.co/datasets/timdettmers/openassistant-guanaco
# Template: https://huggingface.co/datasets/mlabonne/guanaco-llama2?row=0
# Our: https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k
# Load the dataset
oa_dataset = load_dataset('timdettmers/openassistant-guanaco')

# Shuffle the dataset and slice it
oa_dataset = oa_dataset['train'].shuffle(seed=42).select(range(1000))

# Define a function to transform the data
def transform_conversation(example):
    convo_text = example['text']
    segments = convo_text.split('###')

    reformatted_segments = []

    for i in range(1, len(segments) - 1, 2):
        user_text = segments[i].strip().replace('Human:', '').strip()
        ai_text = segments[i+1].strip().replace('Assistant:', '').strip() if i + 1 < len(segments) else ''
        reformatted_segments.append(f'[INST] {user_text} [/INST] {ai_text} ')

    return {'text': ''.join(reformatted_segments)}



# Apply the transformation
transformed_dataset = oa_dataset.map(transform_conversation)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


In [4]:
transformed_dataset[0]

{'text': '[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. '}

In [5]:
# Config model ready for training
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "mlabonne/guanaco-llama2-1k"
finetune_model = "Llama-2-7b-chat-finetune"

# Output folder
output_dir = "./results"

# Num of epochs
num_train_epochs =1

# No change params
use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant = True, "float16", "nf4", False # For quantization
lora_r, lora_alpha, lora_dropout = 64, 16, 0.1
fp16, bf16 =  False, False
per_device_train_batch_size, per_device_eval_batch_size = 4, 4
gradient_accumulation_steps, gradient_checkpointing, max_grad_norm = 1, True, 0.3
learning_rate, weight_decay, optim = 2e-4, 0.001, "paged_adamw_32bit"
lr_scheduler_type, max_steps, warmup_ratio = "cosine", -1, 0.03
group_by_length, save_steps, logging_steps = True, 0, 25
max_seq_length, packing, device_map = None, False, {"": 0}

In [6]:
# Quantize based on given params
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Reduce model memory and computational requirements
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Config base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type
)

In [8]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model, # model se fintune
    train_dataset=transformed_dataset, # Dataset
    peft_config=peft_config,
    processing_class=tokenizer,
    args=training_arguments,
)

In [9]:
trainer.train()

# Save trained LoRA model
trainer.model.save_pretrained(finetune_model)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: hung-huynh0710 (hung-huynh0710-abc). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
25,1.377700
50,1.632800
75,1.213900
100,1.444300
125,1.183600
150,1.368000
175,1.177400
200,1.463500
225,1.161500
250,1.534000


In [10]:
# Test model
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Dime la definición de Psicópata."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] Dime la definición de Psicópata. [/INST] Psicópata es un término que se utiliza para describir a una persona que tiene una personalidad antisocial y que puede tener un trastorno de personalidad antisocial. Es decir, una persona que tiene una falta de empatía, impulsividad, incapacidad para establecer relaciones saludables, tendencia a engañar y manipular a los demás, y que puede tener comportamientos violentes o agresivos. 

Sin embargo, es importante tener en cuenta que el término "psicópata" no es un diagnóstico médico y que los comportamientos y conductas de una persona no son necesariamente una indicación de un trastorno de personalidad antisocial. Además, es importante tener en cuenta que el diagnóstico de un tr


In [11]:
# Merge base model & fine tunned one

final_model = PeftModel.from_pretrained(model, finetune_model)
final_model = final_model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

final_model.save_pretrained("models/final_model.pt")
tokenizer.save_pretrained("models/tokenizer/")


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('models/tokenizer/tokenizer_config.json',
 'models/tokenizer/special_tokens_map.json',
 'models/tokenizer/tokenizer.model',
 'models/tokenizer/added_tokens.json',
 'models/tokenizer/tokenizer.json')